# Bereiten wir das Script vor

In [ ]:
!pip install textract

In [ ]:
#Imports
import os #Files in listen abgreifen
import tqdm #anzeigen, wo wir stehen
import textract
import pandas as pd

## Vorbereitung

In [ ]:
#Als Vorbereitung, machen aus dem Inhalt des Folders eine Liste. Es handelt sich hierbei
#um eine Liste aller pdfs im Ordner "pdfs". Es sind die Swissmedic Recalls (#ImplantFiles)
pdfs = os.listdir('pdfs') 

In [ ]:
pdfs #hier sehen wir den ganzen ordner mit allen pdfs

## Fortschritt anzeigen

In [ ]:
from tqdm import tqdm
for i in tqdm(range(1000)): #100000000000
    elem = i
    
    #er rechnet hier nur einen leeren Loop durch ohne dabei auf ein File zurückzugreiffen. Ist nur als Beispiel

## Ein File auslesen

In [ ]:
text = textract.process("pdfs/"+pdfs[3]) 
#method='pdfminer', encoding='ISO-8859-1'

In [ ]:
#häufig machen Firmen ihre Publikationen als Bild und nicht als Text.(abfotografiert und in PDF getan). 
#Dies können wir mit den aktuellen Mitteln lösen

## Alle Files auslesen

In [ ]:
#hier der For-Loop über alle 82 PDFs.
#wir machen zwei Listen. Eine für die Resultate und eine für die Fehler.
#aus den Elementen (pdf) in der Variable pdfs zieht er den Text raus. Dort stopft diesen wieder in die Liste. 

In [ ]:
lst = []                   
mistakes = []
for pdf in tqdm(pdfs): #pdfs ist die Liste mit den pdfs
    try:
        text = textract.process("pdfs/"+pdf, method='pdfminer', encoding='ISO-8859-1')
        lst.append(text)
    except:
        mistakes.append(pdf)

## Bilder auslesen

In [ ]:
text = textract.process("pdfs/"+pdfs[0], method='tesseract') #Nicht ideal, aber gut

## Alles zusammen

In [ ]:
#Rohe Analyse
for pdf in tqdm(pdfs):
    try:
        text = textract.process("pdfs/"+pdf, method='pdfminer', encoding='ISO-8859-1')
        text = text.decode("ISO-8859-1").replace("\n", " ") #decode benötigts um einen Str draus zu machen. (mit str(text)
                                                            #würde das auch gehen, aber könnte Fehler haben)..replace ist 
                                                            #dass es schöner wird. 
        if len(text) < 40:     #wenn es sehr wenig Text ist, muss es fast ein Bild sein. Deshalb mit dem tesseract behandeln
            text = textract.process("pdfs/"+pdf, method='tesseract',  encoding='ISO-8859-1') 
                                                            #Sprache mit: language='deu', nur definieren, wenn es eine 
                                                            #einzige Sprache ist. Mehrere Sprachen dann weglassen.
            text = text.decode("ISO-8859-1").replace("\n", " ")
            
        mini_dict = {'Text':text,
                     'File':pdf}
        lst.append(mini_dict)
    except:
        mini_dict = {'Text':'Fehlermeldung',
                     'File':pdf}
        lst.append(mini_dict)

In [ ]:
#dann kann man es in ein DataFrame einpacken
pd.DataFrame(lst)

In [ ]:
#Erste Säuberung
df = pd.DataFrame(lst)
def date(elem):
    elem = elem[3:11]
    return elem
df['date'] = df['File'].apply(date)
df['date'] = pd.to_datetime(df['date'], format='%Y%m%d')
df.index = df['date']
#Suchen wir noch nach Implantaten
df['implant'] = df['Text'].str.contains('implant')
df.to_csv('datafile.csv')

In [ ]:
df = pd.DataFrame(lst)

In [ ]:
def length(elem)
    elem=len(elem)
    return elem
    df["Text Länge"]=df["Text"].apply(length) #so könnte man alle PDFs auf deren Länge untersuchen. 